In [1]:
import pandas as pd
import sqlite3

# Load data from CSV files into DataFrames
customers_df = pd.read_csv('customers.csv')
invoices_df = pd.read_csv('invoices.csv')
invoice_lines_df = pd.read_csv('invoice_lines.csv')

# Ingest data into a SQLite database
conn = sqlite3.connect('my_database.db')

customers_df.to_sql('customers', conn, if_exists='replace', index=False)
invoices_df.to_sql('invoices', conn, if_exists='replace', index=False)
invoice_lines_df.to_sql('invoice_lines', conn, if_exists='replace', index=False)


# 1. Number of customers purchasing more than 5 books
query_1 = """
SELECT c.customer_id, c.name, COUNT(il.invoice_id) AS books_purchased
FROM customers c
LEFT JOIN invoices i ON c.customer_id = i.customer_id
LEFT JOIN invoice_lines il ON i.invoice_id = il.invoice_id
GROUP BY c.customer_id, c.name
HAVING COUNT(il.invoice_id) > 5;
"""

# 2. List of customers who never purchased anything
query_2 = """
SELECT c.customer_id, c.name
FROM customers c
LEFT JOIN invoices i ON c.customer_id = i.customer_id
WHERE i.customer_id IS NULL;
"""

# 3. List of books purchased with the users
query_3 = """
SELECT c.name AS customer_name, il.description AS book_description
FROM customers c
JOIN invoices i ON c.customer_id = i.customer_id
JOIN invoice_lines il ON i.invoice_id = il.invoice_id;
"""


# Execute the queries and store the results in DataFrames
result_1 = pd.read_sql_query(query_1, conn)
result_2 = pd.read_sql_query(query_2, conn)
result_3 = pd.read_sql_query(query_3, conn)


# Close the database connection
conn.close()

# Display or save the query results as needed
print("Customers purchasing more than 5 books:")
print(result_1)

print("\nCustomers who never purchased anything:")
print(result_2)

print("\nBooks purchased with the users:")
print(result_3)

Customers purchasing more than 5 books:
Empty DataFrame
Columns: [customer_id, name, books_purchased]
Index: []

Customers who never purchased anything:
   customer_id         name
0            4     Faiz Ma 
1            5  Isham Rais 

Books purchased with the users:
  customer_name book_description
0  Irfan Bakti          Book #1 
1  Jack Smmith          Book #2 
2  Jack Smmith          Book #3 
3  Jack Smmith          Book #1 
4        Nazir          Book #1 
5        Nazir          Book #2 
6  Shanon Teoh          Book #1 
7  Shanon Teoh          Book #3 
